## Evaluate

$ 5x^{3} - 4x^{2}y^{2} + 13xy^{2} + x^{2} -10y = out $

$ v_1 = xx $

$ v_2 = yy $

$ v_3 = 5xv_1 $

$ v_4 = 4v_1v_2 $

$ out -v_3 + v_4 - v_1 + 10y = 13xv_2 $

$ w = \begin{bmatrix}
1 & out & x & y & v_1 & v_2 & v_3 & v_4
\end{bmatrix} $



$ L = \begin{bmatrix}
1 & out & x & y & v_1 & v_2 & v_3 & v_4 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 5 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 4 & 0 & 0 & 0 \\
0 & 0 & 13 & 0 & 0 & 0 & 0 & 0 \\
\end{bmatrix} $


$ R = \begin{bmatrix}
1 & out & x & y & v_1 & v_2 & v_3 & v_4 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
\end{bmatrix} $


$ O = \begin{bmatrix}
1 & out & x & y & v_1 & v_2 & v_3 & v_4 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
0 & 1 & 0 & 10 & -1 & 0 & -1 & 1 \\
\end{bmatrix} $


In [1]:
import galois
import numpy as np

p = 71
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
FP = galois.GF(p)

x = FP(2)
y = FP(3)

v1 = x * x
v2 = y * y
v3 = 5 * x * v1
v4 = 4 * v1 * v2
out = 5*x**3 - 4*x**2*y**2 + 13*x*y**2 + x**2 - 10*y

w = FP([1, out, x, y, v1, v2, v3, v4])

print("w =", w)

R = FP([[0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 5, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 4, 0, 0, 0],
         [0, 0, 13, 0, 0, 0, 0, 0]])

L = FP([[0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0]])

O = FP([[0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 0, 1],
         [0, 1, 0, 10, FP(p - 1), 0, FP(p - 1), 1]])

Lw = np.dot(L, w)
Rw = np.dot(R, w)
Ow = np.dot(O, w)

print("Lw =", Lw)
print("Rw =", Rw)

LwRw = np.multiply(Lw, Rw)

print("Lw * Rw =", LwRw)

print("Ow =     ", Ow)

assert np.all(LwRw == Ow)


w = [  1 104   2   3   4   9  40 144]
Lw = [2 3 4 9 9]
Rw = [ 2  3 10 16 26]
Lw * Rw = [  4   9  40 144 234]
Ow =      [  4   9  40 144 234]


In [2]:
mtxs = [L, R, O]
poly_m = []

for m in mtxs:
    poly_list = []
    for i in range(0, m.shape[1]):
        points_x = FP(np.zeros(m.shape[0], dtype=int))
        points_y = FP(np.zeros(m.shape[0], dtype=int))
        for j in range(0, m.shape[0]):
            points_x[j] = FP(j+1)
            points_y[j] = m[j][i]

        poly = galois.lagrange_poly(points_x, points_y)
        coef = poly.coefficients()[::-1]
        if len(coef) < m.shape[0]:
            coef = np.append(coef, np.zeros(m.shape[0] - len(coef), dtype=int))
        poly_list.append(coef)
    
    poly_m.append(FP(poly_list))

Lp = poly_m[0]
Rp = poly_m[1]
Op = poly_m[2]

print(f'''L
{Lp}
''')

print(f'''R
{Rp}
''')

print(f'''O
{Op}
''')

L
[[                                                                            0
                                                                              0
                                                                              0
                                                                              0
                                                                              0]
 [                                                                            0
                                                                              0
                                                                              0
                                                                              0
                                                                              0]
 [                                                                            5
   9120101196599698009269335727190531286895151833506680976540918411073253539834
    91201011965996980092693357271905

In [3]:
def to_poly(mtx):
    poly_list = []
    for i in range(0, mtx.shape[0]):
        poly_list.append( galois.Poly(mtx[i][::-1]) )
    return poly_list

def print_poly(name, poly_list):
    print(f'\n{name} polynomials:')
    for i in range(0, len(poly_list)):
        print(f'{name}_{i} = {poly_list[i]}')

def evaluate_poly(poly_list, x):
    results = []
    for poly in poly_list:
        results.append(poly(x))
    return results

def print_evaluation(name, results):
    print(f'\n{name} polynomial evaluations:')
    for i in range(0, len(results)):
        print(f'{name}_{i} = {results[i]}')

alpha = FP(2)
beta = FP(3)
tau = FP(20)
gamma = FP(5)
delta = FP(7)

# U = to_poly(Lp)
# print_poly("U", U)

# V = to_poly(Rp)
# print_poly("V", V)

# W = to_poly(Op)
# print_poly("W", W)

# U_beta = [ poly * beta for poly in U ]
# print_poly("βU", U_beta)

# V_alpha = [ poly * alpha for poly in V ]
# print_poly("αV", V_alpha)

# bUaUW = np.add(np.add(U_beta, V_alpha), W)
# print_poly("(βU + αV + W) = K", bUaUW)

# K = evaluate_poly(bUaUW, tau)
# print_evaluation("(βU + αV + W) = K", K)

T = galois.Poly(FP([1, p-1]))
for i in range(2, L.shape[0] + 1):
    T *= galois.Poly(FP([1, p-i]))

print("\nT = ", T)
for i in range(1, L.shape[0] + 2):
    print(f"T({i}) = ", T(i))
    if i == L.shape[0]:
        print("-"*10)

T_tau = T(tau)
print(f"\nT(τ) = {T_tau}")


T =  x^5 + 21888242871839275222246405745257275088548364400416034343698204186575808495602x^4 + 85x^3 + 21888242871839275222246405745257275088548364400416034343698204186575808495392x^2 + 274x + 21888242871839275222246405745257275088548364400416034343698204186575808495497
T(1) =  0
T(2) =  0
T(3) =  0
T(4) =  0
T(5) =  0
----------
T(6) =  120

T(τ) = 1395360


In [12]:
from py_ecc.optimized_bn128 import multiply, G1, G2, add, pairing, neg, normalize

# G1[α]
alpha_G1 = multiply(G1, int(alpha))
# G1[β]
beta_G1 = multiply(G1, int(beta))
# G1[τ^0], G1[τ^1], ..., G1[τ^d-1]
tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, T.degree - 1)]
# G1[βU0(τ) + αV0(τ) + W0(τ)], G1[βU1(τ) + αV1(τ) + W1(τ)], ..., G1[βUd-1(τ) + αVd-1(τ) + Wd-1(τ)]
# cterms_G1 = [multiply(G1, int(k)) for k in K]
# G1[τ^0 * T(τ)], G1[τ^1 * T(τ)], ..., G1[τ^d-2 * T(τ)]
target_G1 = [multiply(G1, int(tau**i * T_tau)) for i in range(0, T.degree - 2)]

# G2[α]
beta_G2 = multiply(G2, int(beta))
# G2[τ^0], G2[τ^1], ..., G2[τ^d-1]
tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, T.degree - 1)]

print(f"\nαG1 = {normalize(alpha_G1)}")
print(f"βG1 = {normalize(beta_G1)}")
print(f"τG1 = {[normalize(point) for point in tau_G1]}")
# print(f"K = {K}")
# print(f"K = {cterms_G1}")
print(f"T(τ)_G1 = {[normalize(point) for point in target_G1]}")

print(f"\nβG2 = {normalize(beta_G2)}")
print(f"τG2 = {[normalize(point) for point in tau_G2]}")


αG1 = (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764)
βG1 = (3353031288059533942658390886683067124040920775575537747144343083137631628272, 19321533766552368860946552437480515441416830039777911637913418824951667761761)
τG1 = [(1, 2), (18947110137775984544896515092961257947872750783784269176923414004072777296602, 12292085037693291586083644966434670280746730626861846747147579999202931064992), (16262199471205794413544947826745938654132104752637586692048329713311590397011, 13296900385261935021718889695689394625708483652039722230815936262285054528714), (21603600070689675766438470661345954782419355034652174505468210225883925863279, 15787091953565760722773063158476721787069408761080596737736006929439659337677)]
T(τ)_G1 = [(1641247283492879903468444805169804215277964208279225379119694118848884481979, 18733245328562972535068072505811612803902036184072974454572123739995203670419), (128252

In [13]:
U = galois.Poly((w @ Lp)[::-1])
V = galois.Poly((w @ Rp)[::-1])
W = galois.Poly((w @ Op)[::-1])

print("U = ", U)
print("V = ", V)
print("W = ", W)

H = (U * V - W) // T
rem = (U * V - W) % T

print("H = ", H)
print("rem = ", rem)

assert rem == 0

U =  11856131555579607412050136445347690672963697383558685269503193934395229601792x^4 + 20064222632519335620392538599819168831169334033714698148390020504361157787655x^3 + 20976232752179305421319472172538221959858849217065366246044112345468483141610x^2 + 12768141675239577212977070018066743801653212566909353367157285775502554955812x + 21888242871839275222246405745257275088548364400416034343698204186575808495601
V =  10944121435919637611123202872628637544274182200208017171849102093287904247809x^4 + 3648040478639879203707734290876212514758060733402672390616367364429301415930x^3 + 10944121435919637611123202872628637544274182200208017171849102093287904247836x^2 + 18240202393199396018538671454381062573790303667013361953081836822146507079635x + 26
W =  12768141675239577212977070018066743801653212566909353367157285775502554955771x^4 + 7296080957279758407415468581752425029516121466805344781232734728858602831936x^3 + 9120101196599698009269335727190531286895151833506680976540918411073253539611x^2 

In [14]:
def evaluate_poly(poly, trusted_points):
    coeff = poly.coefficients()[::-1]

    terms = [multiply(point, int(coeff)) for point, coeff in zip(trusted_points, coeff)]
    evaluation = terms[0]
    for i in range(1, len(terms)):
        evaluation = add(evaluation, terms[i])

    return evaluation

# G1[h0 * τ^0 * T(τ)] + G1[h1 * τ^1 * T(τ)] + ... + G1[hd-2 * τ^d-2 * T(τ)]
HT_G1 = evaluate_poly(H, target_G1)
# G1[u0 * τ^0] + G1[u1 * τ^1] + ... + G1[ud-1 * τ^d-1]
A_G1 = evaluate_poly(U, tau_G1)
# G2[v0 * τ^0] + G2[v1 * τ^1] + ... + G2[vd-1 * τ^d-1]
B_G2 = evaluate_poly(V, tau_G2)
# G1[w0 * τ^0] + G1[w1 * τ^1] + ... + G1[wd-1 * τ^d-1]
Cw_G1 = evaluate_poly(W, tau_G1)

C_G1 = add(Cw_G1, HT_G1)

print(f"\nA_G1 = {normalize(A_G1)}")
print(f"B_G2 = {normalize(B_G2)}")
print(f"C_G1 = {normalize(C_G1)}")

pairing(B_G2, A_G1) == pairing(G2, C_G1)


A_G1 = (8210893835764254590055035960074800790739147451234137252061301547386298490880, 7091255982729168847869635234951214456056946989549772940302350682206850475986)
B_G2 = ((9517836430396510509307306832140942456052425100365705992951478889370236566045, 2949123567199050045864293304213447156856313487229737119230879336811354792596), (3464238454654928685050059532467028586717885732517155631181449831839090947411, 16347716362438583378483045761719058351969666119076212029165303099373302002895))
C_G1 = (20898361232016539553573136171294871683275500864262252320684525864484555367203, 7575134145356869553204761739414395128264207271657993551862134966141472566012)


False